In [ ]:
!apt install -qq chromium-chromedriver

The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 48 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package apparmor.
(Reading database ... 123595 files and directories currently installed.)
Preparing to unpack .../apparmor_3.0.4-2ubuntu2.3_amd64.deb ...
Unpacking apparmor (3.0.4-2ubuntu2.3) ...
Selecting previously unselected package liblzo2-2:amd64.
Preparing to unpack .../liblzo2-2_2.10-2build3_amd64.deb ...
Unpack

In [ ]:
!pip install -qq selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

# Thu thập báo VNExpress

In [ ]:
MAGAZINE_NAME = "vnexpress"
VNEXPRESS_HOME_PAGE = "https://vnexpress.net/"

## Thu thập các thể loại

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
driver.get(VNEXPRESS_HOME_PAGE)

In [ ]:
# Lấy ra tất cả các thể loại báo
vnexpress_cats = []

row_menu = driver.find_element(by=By.CLASS_NAME, value='row-menu')
cat_menus = row_menu.find_elements(by=By.CLASS_NAME, value='cat-menu')

for cat in cat_menus:
  # Lấy ra tiêu đề và url
  cat_name = cat.find_element(by=By.TAG_NAME, value='a').get_attribute('title')
  url = cat.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
  vnexpress_cats.append({"cat_name" : cat_name, "url" : url})

In [ ]:
vnexpress_cats, len(vnexpress_cats)

([{'cat_name': 'Thời sự', 'url': 'https://vnexpress.net/thoi-su'},
  {'cat_name': 'Góc nhìn', 'url': 'https://vnexpress.net/goc-nhin'},
  {'cat_name': 'Thế giới', 'url': 'https://vnexpress.net/the-gioi'},
  {'cat_name': 'Video', 'url': 'https://video.vnexpress.net/'},
  {'cat_name': 'Podcasts', 'url': 'https://vnexpress.net/podcast'},
  {'cat_name': 'Kinh doanh', 'url': 'https://vnexpress.net/kinh-doanh'},
  {'cat_name': 'Bất động sản', 'url': 'https://vnexpress.net/bat-dong-san'},
  {'cat_name': 'Khoa học', 'url': 'https://vnexpress.net/khoa-hoc'},
  {'cat_name': 'Giải trí', 'url': 'https://vnexpress.net/giai-tri'},
  {'cat_name': 'Thể thao', 'url': 'https://vnexpress.net/the-thao'},
  {'cat_name': 'Pháp luật', 'url': 'https://vnexpress.net/phap-luat'},
  {'cat_name': 'Giáo dục', 'url': 'https://vnexpress.net/giao-duc'},
  {'cat_name': 'Sức khỏe', 'url': 'https://vnexpress.net/suc-khoe'},
  {'cat_name': 'Đời sống', 'url': 'https://vnexpress.net/doi-song'},
  {'cat_name': 'Du lịch', 'u

In [ ]:
driver.close()

## Thu thập đường dẫn đến các bài báo cho từng thể loại

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# Số lượng bài báo mỗi thể loại
NUM_ARTICLE_PER_CAT = 400
# Đường dẫn lưu đường link các bài
VNEXPRESS_DATA_URL_FILE = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/vnexpress_url.json'
# Bỏ qua các thể
VNEXPRESS_EXCLUDING_CATEGORIES = ["Góc nhìn", "Video", "Podcasts", "Ý kiến", "Tâm sự", "Thư giãn"]
# Bổ sung cài đặt
chrome_options.page_load_strategy = 'normal'

In [ ]:
crawled_url = set()

def crawl_each_category(driver, category_url):
  """
  Function lấy ra NUM_ARTICLE_PER_CAT mỗi loại
  """
  all_urls = set()
  url = category_url

  while(len(all_urls) < NUM_ARTICLE_PER_CAT):
    driver.get(url)
    # Lấy ra tất cả title bài báo trong page
    title_news = driver.find_elements(by=By.CLASS_NAME, value='title-news')
    for title in title_news:
      try:
        url_new = title.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
        if url_new not in crawled_url and url_new.startswith(VNEXPRESS_HOME_PAGE):
          all_urls.add(url_new)
          crawled_url.add(url_new)
      except StaleElementReferenceException:
          continue
      except NoSuchElementException:
          print(f"NoSuchElementException at {url}")
          continue
    # trang tiếp theo
    url = driver.find_element(by=By.CLASS_NAME, value='next-page').get_attribute('href')
  return all_urls

In [ ]:
saved_cats = {}
for cat in vnexpress_cats:
  cat_name = cat['cat_name']
  url = cat['url']
  if cat_name not in VNEXPRESS_EXCLUDING_CATEGORIES:
    print(f'You are at {cat_name}')
    all_urls = crawl_each_category(driver, url)
    saved_cats[cat_name] = list(all_urls)
driver.close()

You are at Thời sự
You are at Thế giới
You are at Kinh doanh
You are at Bất động sản
You are at Khoa học
You are at Giải trí
You are at Thể thao
You are at Pháp luật
You are at Giáo dục
You are at Sức khỏe
You are at Đời sống
You are at Du lịch
You are at Số hóa
You are at Xe


In [ ]:
with open(VNEXPRESS_DATA_URL_FILE, 'w', encoding='utf-8') as f:
   json.dump(saved_cats, f, indent=4, ensure_ascii=False)

## Thu thập metadata các bài báo từng thể loại

In [ ]:
# Đường dẫn lưu đường link các bài
VNEXPRESS_DATA_URL_FILE = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/vnexpress_url.json'
# Cài đặt strategy để bỏ qua ảnh
chrome_options.page_load_strategy = "eager"
# Folder chứa metadata từng thể loại báo
DATA_FOLDER = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/vnexpress'

In [ ]:
with open(VNEXPRESS_DATA_URL_FILE, 'r', encoding='utf-8') as f:
  url_data = json.load(f)
print(f"Số thể loại: {len(url_data)}")
print(f"Số bài báo: {sum([len(v) for v in url_data.values()])}")

Số thể loại: 14
Số bài báo: 5815


In [ ]:
def get_content_metadata(driver, article_url):
    """
    Function bóc tách các trường cần thu thập trong 1 bài báo
    """
    driver.get(article_url)
    # Tiêu đề
    title = driver.find_element(by=By.CLASS_NAME, value='title-detail').text.strip()
    # Description
    description = driver.find_element(by=By.CLASS_NAME, value='description').text.strip()
    # Thể loại
    lis_cats = driver.find_element(by=By.CLASS_NAME, value='breadcrumb').find_elements(by=By.TAG_NAME, value='li')
    cat = lis_cats[0].text.strip() if len(lis_cats) > 0 else None
    subcat = lis_cats[1].text.strip() if len(lis_cats) > 1 else None
    # Lấy ra ngày phát hành
    published_date = driver.find_element(by=By.CSS_SELECTOR, value='[itemprop="datePublished"]').get_attribute('content').strip()

    # Thu thập content bài báo
    # Locate phần viết content
    article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
    # Lấy hết các đầu mục con của bài báo
    children = article.find_elements(by=By.XPATH, value="./*")

    contents = []
    author = "Unknown"

    # Check có phải dạng slide show hay không
    is_slide_show = False
    for idx, child in enumerate(children):
        text = child.text.strip()
        # Nếu mà element right align --> có thể là tác giả
        if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
            author = text
        elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
            # If center
            if len(text):
                if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Chỉ lấy caption của figure
        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if len(text):
                if len(text) <= 100: # nếu mà len <= 100 --> add thêm [] xung quanh
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Nếu mà là slide show thì nó giống figure
        elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
            is_slide_show = True # slideshow
            if len(text):
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Bỏ qua table
        elif child.tag_name == "table":
            pass

    if is_slide_show:
        author = text

    # Nếu mà vẫn chưa thấy author thì tìm bằng tag
    if author == "Unknown":
        try:
            author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author')]").text
        except:
            pass

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents), # join các đoạn bằng \n
        "metadata": {
            "cat": cat,
            "subcat": subcat,
            "published_date": published_date,
            "author": author
        }
    }


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
demo_url = list(url_data.values())[0][0]
print(json.dumps(get_content_metadata(driver, demo_url), indent=4, ensure_ascii=False))
driver.close()

{
    "url": "https://vnexpress.net/dai-vtc-ky-niem-20-nam-thanh-lap-4780701.html",
    "title": "Đài VTC kỷ niệm 20 năm thành lập",
    "description": "Từ 50 nhân sự đầu tiên, \"cánh sóng\" VTC đã vươn khắp đất nước với 15 kênh và một báo điện tử, đáp ứng nhu cầu của hàng triệu khán giả.",
    "content": "Ngày 12/8, tại Hà Nội, Đài Truyền hình kỹ thuật số VTC tổ chức kỷ niệm 20 năm thành lập. Ban Biên tập Truyền hình Kỹ thuật số VTC - tiền thân của Đài VTC - được ra mắt ngày 19/8/2004. Với 50 nhân sự ban đầu, chương trình truyền hình đầu tiên được lên sóng sau hai tuần.\n\"Truyền hình số VTC ra đời gây tiếng vang vì hình rõ nét, tiếng trung thực, truyền hình tốt và xa, ưu điểm lớn nhất là lưu trữ được tín hiệu\", nguyên Bộ trưởng Bưu chính Viễn thông Đỗ Trung Tá nhận xét.\nÔng Lưu Vũ Hải, nguyên Phó tổng giám đốc Đài VTC, sau này có thời gian là Cục trưởng Phát thanh Truyền hình (Bộ Thông tin và Truyền thông) từng nói: \"VTC ra đời được xã hội đón nhận nồng nhiệt và từng bước thuyết p

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():
  print(f"Thu thập dữ liệu thể loại {cat}...")
  cat_data = []
  for url in tqdm(urls, desc=f"Processing {cat}", unit="article"):
    try:
      cat_data.append(get_content_metadata(driver, url))
    except (StaleElementReferenceException, NoSuchElementException) as e:
            print(f"Bug at url: {url}, with ElementException")
            driver.refresh()
            continue
  name_file_cat = cat.lower().replace(" ", "_") + ".json"
  with open(os.path.join(DATA_FOLDER, name_file_cat), 'w', encoding='utf-8') as f:
    json.dump(cat_data, f, indent=4, ensure_ascii=False)
driver.close()


Thu thập dữ liệu thể loại Sức khỏe...


Processing Sức khỏe:   0%|          | 0/421 [00:00<?, ?article/s]

Bug at url: https://vnexpress.net/hang-nghin-nguoi-mac-benh-tieu-hoa-hau-covid-4579633.html, with ElementException


Processing Sức khỏe:   3%|▎         | 14/421 [06:53<49:25,  7.29s/article]

Bug at url: https://vnexpress.net/vi-sao-duoc-tang-tien-nguoi-dan-the-gioi-cung-khong-muon-sinh-con-4779304.html, with ElementException


...

Processing Xe:  93%|█████████▎| 373/400 [54:49<03:34,  7.95s/article]

Bug at url: https://vnexpress.net/oto-xe-may/v-car/doanh-so/5-xe-gam-cao-ban-nhieu-nhat-thang-6-lan-dau-cua-ford-territory-4769156.html, with ElementException


Processing Xe:  94%|█████████▍| 376/400 [55:18<03:35,  8.96s/article]

Bug at url: https://vnexpress.net/nen-mua-i10-hay-cho-mua-xe-dien-vf-3-4765726.html, with ElementException


Processing Xe: 100%|██████████| 400/400 [59:04<00:00,  8.86s/article]


## Thu thập Hot news

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
VNEXPRESS_HOT_NEWS_PAGE = 'https://vnexpress.net/tin-nong'

In [ ]:
# Cài đặt bổ sung strategy
chrome_options.page_load_strategy = 'normal'

In [ ]:
hot_news_urls = set()
driver.get(VNEXPRESS_HOT_NEWS_PAGE)
# Lấy ra tất cả title bài báo trong page
title_news = driver.find_elements(by=By.CLASS_NAME, value='item-news-common')
for title in title_news:
  try:
    url_new = title.find_element(by=By.CLASS_NAME, value='title-news').find_element(by=By.TAG_NAME, value='a').get_attribute('href')
    if url_new.startswith(VNEXPRESS_HOME_PAGE):
      hot_news_urls.add(url_new)
  except StaleElementReferenceException:
      continue
  except NoSuchElementException:
      print(f"NoSuchElementException at {url}")
      continue
driver.close()

In [ ]:
hot_news_urls, len(hot_news_urls)

({'https://vnexpress.net/111-nguoi-doi-no-thue-kieu-xa-hoi-den-cho-cac-ngan-hang-linh-an-4787529.html',
  'https://vnexpress.net/alcaraz-thua-soc-o-vong-hai-my-mo-rong-4787485.html',
  'https://vnexpress.net/bao-shanshan-tan-cong-nhat-5-nguoi-chet-4787351.html',
  'https://vnexpress.net/ben-trong-nha-may-xu-ly-nuoc-thai-lon-nhat-nuoc-o-tp-hcm-4787524.html',
  'https://vnexpress.net/bo-doi-ra-soat-hien-truong-hai-vu-no-o-nghe-an-4787440.html',
  'https://vnexpress.net/con-tin-ke-ca-mo-khong-thuoc-me-o-gaza-4787385.html',
  'https://vnexpress.net/gia-tram-song-gui-tin-nhan-rac-o-tp-hcm-4787495.html',
  'https://vnexpress.net/lanh-dao-dang-nha-nuoc-vao-lang-vieng-chu-tich-ho-chi-minh-4787516.html',
  'https://vnexpress.net/lao-cai-thai-nguyen-ngap-sau-do-mua-lon-4787456.html',
  'https://vnexpress.net/lien-danh-han-quoc-tu-van-quan-ly-khai-thac-san-bay-long-thanh-4787581.html',
  'https://vnexpress.net/nguoi-chan-duong-danh-ban-hoc-cua-con-bi-phat-tu-4787386.html',
  'https://vnexpress.ne

In [ ]:
# Cài đặt strategy để bỏ qua ảnh
chrome_options.page_load_strategy = "eager"
# Folder chứa metadata từng thể loại báo
VNEXPRESS_DATA_FOLDER = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/vnexpress'

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
hot_news_data = []
for url in tqdm(hot_news_urls, desc=f"Processing Hot news", unit="article"):
  try:
    hot_news_data.append(get_content_metadata(driver, url))
  except (StaleElementReferenceException, NoSuchElementException) as e:
          print(f"Bug at url: {url}, with ElementException")
          driver.refresh()
          continue
with open(os.path.join(VNEXPRESS_DATA_FOLDER, 'hot_news.json'), 'w', encoding='utf-8') as f:
  json.dump(hot_news_data, f, indent=4, ensure_ascii=False)
driver.close()

Processing Hot news:  50%|█████     | 10/20 [03:14<03:22, 20.24s/article]

Bug at url: https://vnexpress.net/vu-roi-tiem-kich-f-16-giang-don-vao-quan-doi-ukraine-4787422.html, with ElementException


Processing Hot news:  70%|███████   | 14/20 [04:41<02:06, 21.14s/article]

Bug at url: https://vnexpress.net/nguoi-chan-duong-danh-ban-hoc-cua-con-bi-phat-tu-4787386.html, with ElementException


Processing Hot news:  90%|█████████ | 18/20 [05:59<00:38, 19.35s/article]

Bug at url: https://vnexpress.net/lao-cai-thai-nguyen-ngap-sau-do-mua-lon-4787456.html, with ElementException


Processing Hot news: 100%|██████████| 20/20 [06:28<00:00, 19.42s/article]


# Thu thập báo Vietnamnet

In [ ]:
VIETNAMNET_HOME_PAGE = 'https://vietnamnet.vn/'

## Thu thập các thể loại

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
driver.get(VIETNAMNET_HOME_PAGE)

In [ ]:
# Lấy hết tất cả thể loại từ đây
vietnamnet_cats = []
# container chứa tất cả thể loại
container = driver.find_element(by = By.CLASS_NAME, value='header_submenu-content-list')
# lấy tất cả thể loại
cat_menus = container.find_elements(by = By.TAG_NAME, value='li')
for cat_menu in cat_menus:
    cat = cat_menu.find_element(by = By.TAG_NAME, value='a').get_attribute('title')
# bỏ qua logo-premium, logo-htvn
    if(cat not in ['premium vietnamnet', 'Hành trình Việt Nam']):
        href = cat_menu.find_element(by = By.TAG_NAME, value='a').get_attribute('href')
        vietnamnet_cats.append({'cat_name': cat, 'url': href})


In [ ]:
vietnamnet_cats, len(vietnamnet_cats)

([{'cat_name': 'Chính trị', 'url': 'https://vietnamnet.vn/thoi-su/chinh-tri'},
  {'cat_name': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'},
  {'cat_name': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'},
  {'cat_name': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'},
  {'cat_name': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'},
  {'cat_name': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'},
  {'cat_name': 'Giải trí', 'url': 'https://vietnamnet.vn/giai-tri'},
  {'cat_name': 'Văn hóa', 'url': 'https://vietnamnet.vn/van-hoa'},
  {'cat_name': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'},
  {'cat_name': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'},
  {'cat_name': 'Thông tin và Truyền thông',
   'url': 'https://vietnamnet.vn/thong-tin-truyen-thong'},
  {'cat_name': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'},
  {'cat_name': 'Ô tô xe máy', 'url': 'https://vietnamnet.vn/oto-xe-may'},
  {'cat_name': 'Bất động sản', 'url': 'https://vietn

## Thu thập đường dẫn của từng thể loại báo

In [ ]:
NUM_ARTICLES_PER_CAT = 400
# Đường dẫn lưu đường link các bài
VIETNAMNET_DATA_URL_FILE = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/vietnamnet_url.json'
# Bỏ qua các thể loại
EXCLUDING_CATEGORIES_VIETNAMNET = ["Đính chính", "Giảm nghèo bền vững", "English", "Toàn văn"]
# Cài đặt load strategy là normal
chrome_options.page_load_strategy = 'normal'

In [ ]:
crawled_urls = set()

def crawl_each_category(driver, category_url):
  """
  Function lấy ra NUM_ARTICLE_PER_CAT mỗi loại
  """
  all_urls = set()
  url = category_url
  while(len(all_urls) < NUM_ARTICLES_PER_CAT):
    driver.get(url)
    vnn_titles = driver.find_elements(by=By.CLASS_NAME, value='vnn-title')
    for title in vnn_titles:
      try:
        url_new = title.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
        if url_new not in crawled_urls and url_new.startswith(VIETNAMNET_HOME_PAGE):
          all_urls.add(url_new)
          crawled_urls.add(url_new)
      except StaleElementReferenceException:
          continue
      except NoSuchElementException:
          print(f"NoSuchElementException at {url}")
          continue
      # next-page
    url = driver.find_element(by=By.CLASS_NAME, value='pagination-next').find_element(by=By.TAG_NAME, value='a').get_attribute('href')
  return all_urls

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
saved_cats = {}
for cat in vietnamnet_cats:
  cat_name = cat['cat_name']
  url = cat['url']
  if cat_name not in EXCLUDING_CATEGORIES_VIETNAMNET:
    print(f'You are at {cat_name}')
    all_urls = crawl_each_category(driver, url)
    saved_cats[cat_name] = list(all_urls)
driver.close()

You are at Chính trị
You are at Kinh doanh
You are at Thể thao
You are at Thế giới
You are at Giáo dục
You are at Giải trí
You are at Văn hóa
You are at Đời sống
You are at Sức khỏe
You are at Thông tin và Truyền thông
You are at Pháp luật
You are at Ô tô xe máy
You are at Bất động sản
You are at Du lịch
You are at Bạn đọc
You are at Tuần Việt Nam
You are at Công nghiệp hỗ trợ
You are at Bảo vệ người tiêu dùng
You are at Thị trường tiêu dùng
You are at Dân tộc - Tôn giáo
You are at Nông thôn mới
You are at Dân tộc thiểu số và miền núi
You are at Nội dung chuyên đề


In [ ]:
with open(VIETNAMNET_DATA_URL_FILE, 'w', encoding='utf-8') as f:
   json.dump(saved_cats, f, indent=4, ensure_ascii=False)

## Thu thập metadata cho từng thể loại

In [ ]:
# Cài đặt strategy để bỏ qua ảnh
chrome_options.page_load_strategy = "eager"
# Folder chứa metadata từng thể loại báo
VIETNAMNET_DATA_FOLDER = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/vietnamnet'

In [ ]:
with open(VIETNAMNET_DATA_URL_FILE, 'r', encoding='utf-8') as f:
  url_data = json.load(f)
len(url_data),sum([len(v) for v in url_data.values()])

In [ ]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    title = driver.find_element(by = By.CLASS_NAME, value = 'content-detail-title').text

    # Thu thập description
    description = driver.find_element(by = By.CLASS_NAME, value = 'content-detail-sapo').text

    # Thu thập thể loại
    lis_cats = driver.find_element(by = By.CLASS_NAME, value='bread-crumb-detail').find_elements(by=By.TAG_NAME, value='li')
    main_cat = lis_cats[1].find_element(by = By.TAG_NAME, value='a').text if len(lis_cats) > 1 else None
    sub_cat = lis_cats[2].find_element(by = By.TAG_NAME, value='a').text if len(lis_cats) > 2 else None

    # Thu thập published date
    script_tag = driver.find_elements(by=By.XPATH, value='//script[@type="application/ld+json"]')
    content_json = script_tag[1].get_attribute('innerHTML')
    content_json = ''.join(c for c in content_json if c.isprintable())
    data = json.loads(content_json)
    published_date = data['datePublished']

    # Thu thập content bài báo
    article = driver.find_element(by=By.CLASS_NAME, value='maincontent')
    children = article.find_elements(by=By.XPATH, value='./*')
    contents = []

    for idx, child in enumerate(children):
        text = child.text.strip()
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph
            contents.append(text)

        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if(len(text)):
                if(len(text)) > 100:
                    contents.append(text)
                else:
                    contents.append(f'[{text}]')

        elif child.tag_name == "table":
            pass

    # Thu thập tác giả
    is_author_maybe_last = False
    try:
        author = driver.find_element(by=By.CLASS_NAME, value='article-detail-author__info').text
    except NoSuchElementException:
        is_author_maybe_last = True

    # Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 50
    if is_author_maybe_last:
        if len(contents):
            author = contents.pop()
        else:
            author = "Uknown"


    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": published_date,
            "author": author
        }
    }


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
demo_url = list(url_data.values())[0][0]
print(json.dumps(get_content_metadata(driver, demo_url), indent=4, ensure_ascii=False))
driver.close()

{
    "url": "https://vietnamnet.vn/ong-nguyen-hoa-binh-tach-an-de-cac-chau-khong-bi-am-anh-boi-tuoi-tho-pham-toi-2311387.html",
    "title": "Ông Nguyễn Hòa Bình: Tách án để các cháu không bị ám ảnh bởi tuổi thơ phạm tội",
    "description": "Chánh án TAND Tối cao Nguyễn Hòa Bình cho rằng, nếu không tách vụ án có người chưa thành niên phạm tội mà để vụ án chung, xét xử công khai thì sẽ ảnh hưởng đến con đường hoàn lương, các cháu sẽ luôn bị ám ảnh bởi một tuổi thơ phạm tội.",
    "content": "Chiều ngày 13/8, Ủy ban Thường vụ Quốc hội cho ý kiến về một số vấn đề lớn trong giải trình, tiếp thu, chỉnh lý dự thảo Luật Tư pháp người chưa thành niên.\nMột trong những nội dung đáng chú ý được các đại biểu quan tâm là việc tách vụ án có người chưa thành niên phạm tội.\nTách vụ án là cần thiết\nChủ nhiệm Ủy ban Tư pháp Lê Thị Nga cho biết, quy định này vẫn có ý kiến khác nhau. Quá trình chỉnh lý, trên cơ sở nghiên cứu, thảo luận kỹ lưỡng các chính sách mới của dự thảo luật, đại diện các cơ qua

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
for cat, urls in url_data.items():
  print(f"Thu thập dữ liệu thể loại {cat}...")
  cat_data = []
  for url in tqdm(urls, desc=f"Processing {cat}", unit="article"):
    try:
      cat_data.append(get_content_metadata(driver, url))
    except (StaleElementReferenceException, NoSuchElementException) as e:
            print(f"Bug at url: {url}, with ElementException")
            driver.refresh()
    except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue
  name_file_cat = cat.lower().replace(" ", "_") + ".json"
  with open(os.path.join(VIETNAMNET_DATA_FOLDER, name_file_cat), 'w', encoding='utf-8') as f:
    json.dump(cat_data, f, indent=4, ensure_ascii=False)
driver.close()

Thu thập dữ liệu thể loại Văn hóa...


Processing Văn hóa:   0%|          | 5/1117 [00:28<1:40:48,  5.44s/article]

Bug at url: https://vietnamnet.vn/bat-mi-ve-nha-su-noi-tieng-co-sach-hut-trieu-view-2276110.html, with ElementException


Processing Văn hóa:   3%|▎         | 37/1117 [02:27<1:02:50,  3.49s/article]

Bug at url: https://vietnamnet.vn/ngoi-sao-nhac-do-lung-danh-tuyet-thanh-tuoi-83-song-mot-minh-o-nha-thue-20m2-2274163.html, with ElementException


...

Processing Văn hóa:  78%|███████▊  | 868/1117 [54:33<16:49,  4.05s/article]

Bug at url: https://vietnamnet.vn/nhung-buc-anh-tuyet-pham-ve-hoa-sen-2295469.html, with ElementException


Processing Văn hóa:  80%|████████  | 896/1117 [56:14<13:21,  3.63s/article]

Bug at url: https://vietnamnet.vn/buoc-dot-pha-manh-me-cua-dieu-con-mai-2024-2309745.html, with ElementException


Processing Văn hóa: 100%|██████████| 1117/1117 [1:10:13<00:00,  3.77s/article]


## Thu thập Hot news

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
VIETNAMNET_HOT_NEWS_PAGE = 'https://vietnamnet.vn/tin-moi-nong'

In [ ]:
# Cài đặt bổ sung strategy
chrome_options.page_load_strategy = 'normal'

In [ ]:
hot_news_urls = set()
driver.get(VIETNAMNET_HOT_NEWS_PAGE)
# Lấy ra tất cả title bài báo trong page
title_news = driver.find_elements(by=By.CLASS_NAME, value='vnn-title')
for title in title_news:
  try:
    url_new = title.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
    if url_new.startswith(VIETNAMNET_HOME_PAGE):
      hot_news_urls.add(url_new)
  except StaleElementReferenceException:
      continue
  except NoSuchElementException:
      print(f"NoSuchElementException at {url}")
      continue
driver.close()

In [ ]:
hot_news_urls, len(hot_news_urls)

({'https://vietnamnet.vn/2-vu-no-lien-tiep-4-nguoi-thuong-vong-so-tan-khan-6-ho-dan-de-ra-pha-bom-min-2317018.html',
  'https://vietnamnet.vn/ba-khong-khi-an-hai-san-2316549.html',
  'https://vietnamnet.vn/bac-bo-hung-mua-nhu-trut-sam-set-oanh-tac-suot-sang-30-8-2317063.html',
  'https://vietnamnet.vn/bat-tong-giam-doc-cong-ty-cp-sua-ha-lan-nguyen-trung-vuong-vi-san-xuat-hang-gia-2317015.html',
  'https://vietnamnet.vn/bo-ba-sedan-co-d-toyota-camry-honda-accord-mazda6-gia-chua-den-200-trieu-dong-2316526.html',
  'https://vietnamnet.vn/danh-sach-cac-truong-dai-hoc-cong-lap-xet-tuyen-bo-sung-2316567.html',
  'https://vietnamnet.vn/dau-gia-dat-con-cuong-no-cua-dat-2316916.html',
  'https://vietnamnet.vn/di-giua-troi-ruc-ro-tap-23-chai-ghen-tuong-long-lon-voi-thai-2316949.html',
  'https://vietnamnet.vn/hang-nghin-can-ho-bo-hoang-tai-ha-noi-2290445.html',
  'https://vietnamnet.vn/hoa-hau-ky-duyen-len-tieng-ve-on-ao-doc-sach-2317076.html',
  'https://vietnamnet.vn/khach-tay-an-3-bua-no-ne-o

In [ ]:
# Cài đặt strategy để bỏ qua ảnh
chrome_options.page_load_strategy = "eager"
# Folder chứa metadata từng thể loại báo
VIETNAMNET_DATA_FOLDER = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/vietnamnet'

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
hot_news_data = []
for url in tqdm(hot_news_urls, desc=f"Processing Hot news", unit="article"):
  try:
    hot_news_data.append(get_content_metadata(driver, url))
  except (StaleElementReferenceException, NoSuchElementException) as e:
          print(f"Bug at url: {url}, with ElementException")
          driver.refresh()
          continue
with open(os.path.join(VIETNAMNET_DATA_FOLDER, 'hot_news.json'), 'w', encoding='utf-8') as f:
  json.dump(hot_news_data, f, indent=4, ensure_ascii=False)
driver.close()

Processing Hot news: 100%|██████████| 27/27 [02:13<00:00,  4.93s/article]


# Thu thập từ báo dân trí

In [ ]:
DANTRI_HOME_PAGE = 'https://dantri.com.vn/'

## Thu thập các thể loại

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
driver.get(DANTRI_HOME_PAGE)

In [ ]:
# Choose the menu
menu = driver.find_element(by=By.CLASS_NAME, value='menu-more')
menu.click()

In [ ]:
# Lấy hết tất cả thể loại từ đây
dantri_cats = []

nf_menu = driver.find_element(by=By.CLASS_NAME, value='nf-menu')
cat_menus = nf_menu.find_elements(by=By.XPATH, value='./*')
for cat_menu in cat_menus:
    cat = cat_menu.find_element(by=By.TAG_NAME, value='a').text
    url = cat_menu.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
    dantri_cats.append({'cat_name': cat, 'url': url})

In [ ]:
dantri_cats, len(dantri_cats)

([{'cat_name': 'XÃ HỘI', 'url': 'https://dantri.com.vn/xa-hoi.htm'},
  {'cat_name': 'THẾ GIỚI', 'url': 'https://dantri.com.vn/the-gioi.htm'},
  {'cat_name': 'KINH DOANH', 'url': 'https://dantri.com.vn/kinh-doanh.htm'},
  {'cat_name': 'BẤT ĐỘNG SẢN',
   'url': 'https://dantri.com.vn/bat-dong-san.htm'},
  {'cat_name': 'THỂ THAO', 'url': 'https://dantri.com.vn/the-thao.htm'},
  {'cat_name': 'VIỆC LÀM',
   'url': 'https://dantri.com.vn/lao-dong-viec-lam.htm'},
  {'cat_name': 'NHÂN ÁI', 'url': 'https://dantri.com.vn/tam-long-nhan-ai.htm'},
  {'cat_name': 'SỨC KHỎE', 'url': 'https://dantri.com.vn/suc-khoe.htm'},
  {'cat_name': 'GIẢI TRÍ', 'url': 'https://dantri.com.vn/giai-tri.htm'},
  {'cat_name': 'XE ++', 'url': 'https://dantri.com.vn/o-to-xe-may.htm'},
  {'cat_name': 'SỨC MẠNH SỐ', 'url': 'https://dantri.com.vn/suc-manh-so.htm'},
  {'cat_name': 'GIÁO DỤC', 'url': 'https://dantri.com.vn/giao-duc.htm'},
  {'cat_name': 'AN SINH', 'url': 'https://dantri.com.vn/an-sinh.htm'},
  {'cat_name': 'P

## Thu thập đường dẫn các bài báo cho từng thể loại


In [ ]:
# Số lượng bài báo mỗi thể loại
NUM_ARTICLES_PER_CAT = 400
# Đường dẫn lưu đường link các bài
DANTRI_DATA_URL_FILE = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/dantri_url.json'
# Cài đặt bổ sung strategy
chrome_options.page_load_strategy = 'normal'

In [ ]:
crawled_url = set()

def crawl_category_url(driver, category_url):
    all_urls = set()
    url = category_url
    while(len(all_urls) < NUM_ARTICLES_PER_CAT):
        driver.get(url)
        article_titles = driver.find_elements(by=By.CLASS_NAME, value='article-title')
        for title in article_titles:
            try:
                url_new = title.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
                if url_new.startswith(DANTRI_HOME_PAGE) and url_new not in crawled_url:
                    all_urls.add(url_new)
                    crawled_url.add(url_new)
            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                print(f"NoSuchElementException at {url_new}")
                continue
        try:
            url = driver.find_element(by=By.CLASS_NAME, value='next').get_attribute('href')
        except NoSuchElementException:
            break
    return all_urls

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
saved_cats = {}

for cat in dantri_cats:
    cat_name = cat['cat_name']
    url = cat['url']
    print(f"You are at {cat_name}.")
    urls = crawl_category_url(driver, url)
    saved_cats[cat_name] = list(urls)
driver.close()

You are at XÃ HỘI.
You are at THẾ GIỚI.
You are at KINH DOANH.
You are at BẤT ĐỘNG SẢN.
You are at THỂ THAO.
You are at VIỆC LÀM.
You are at NHÂN ÁI.
You are at SỨC KHỎE.
You are at GIẢI TRÍ.
You are at XE ++.
You are at SỨC MẠNH SỐ.
You are at GIÁO DỤC.
You are at AN SINH.
You are at PHÁP LUẬT.
You are at DU LỊCH.
You are at ĐỜI SỐNG.
You are at TÌNH YÊU.
You are at KHOA HỌC - CÔNG NGHỆ.


In [ ]:
with open(DANTRI_DATA_URL_FILE, 'w', encoding='utf-8') as f:
   json.dump(saved_cats, f, indent=4, ensure_ascii=False)

## Thu thập metadata các bài báo từng thể loại

In [ ]:
# Cài đặt strategy để bỏ qua ảnh
chrome_options.page_load_strategy = "eager"
# Folder chứa metadata từng thể loại báo
DANTRI_DATA_FOLDER = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/dantri'

In [ ]:
with open(DANTRI_DATA_URL_FILE, 'r', encoding='utf-8') as f:
  url_data = json.load(f)
len(url_data), sum([len(v) for v in url_data.values()])

(18, 7054)

In [ ]:
# Class Exception for ignore emagazine
class MagazineArticleException(Exception):
    pass

def get_content_metadata(driver, article_url):
    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)


    # Tạm thời bỏ qua emagazine
    is_emagazine = True
    try:
        driver.find_element(by=By.CLASS_NAME, value="emagazine")
    except NoSuchElementException:
        is_emagazine = False

    if is_emagazine:
        raise MagazineArticleException("We ignore MagazineArticle")

    # Thu thập title
    title = driver.find_element(by=By.CLASS_NAME, value='title-page').text

    # Thu thập description
    description = driver.find_element(by=By.CLASS_NAME, value='singular-sapo').text
    # Bỏ (dân trí) ở đầu
    DANTRI_PHRASE = "(Dân trí) - "
    if DANTRI_PHRASE in description:
        description = description.replace(DANTRI_PHRASE, "")

    # Thu thập thể loại
    lis_cats = driver.find_elements(by=By.CLASS_NAME, value='dt-font-Inter')
    main_cat = None
    sub_cat = None
    for cat in lis_cats:
        if cat.tag_name == 'li':
            a = cat.find_element(by=By.TAG_NAME, value='a')
            if a is not None and a.get_attribute('data-content-piece') == 'article-breadcrumb-position_1':
                main_cat = a.text
            if a is not None and a.get_attribute('data-content-piece') == 'article-breadcrumb-position_2':
                sub_cat = a.text
                break

    # Thu thập published date
    script_tag = driver.find_elements(by=By.XPATH, value='//script[@type="application/ld+json"]')
    content_json = script_tag[1].get_attribute('innerHTML')
    data = json.loads(content_json)
    published_date = data.get('datePublished')

    # Thu thập người viết
    author = "Unknown"
    try:
        author = driver.find_element(by=By.CLASS_NAME, value='author-name').text
    except:
        pass

    # Thu thập content bài báo
    articles = driver.find_element(by=By.CLASS_NAME, value='singular-content')
    children = articles.find_elements(by=By.XPATH, value='./*')
    contents = []

    for idx, child in enumerate(children):
        text = child.text.strip()
        # right align
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph
            # If center
            contents.append(text)
            ####
        elif child.tag_name == "ol":
            for li in child.find_elements(by=By.TAG_NAME, value='li'):
                text = li.text.strip()
                if len(text):
                    contents.append(text)
        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if len(text):
                if len(text) < 100:
                    contents.append(f'[{text}]')
                else:
                    contents.append(text)
        elif child.tag_name == "table":
            pass

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": published_date,
            "author": author
        }
    }


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
demo_url = list(url_data.values())[0][0]
print(json.dumps(get_content_metadata(driver, demo_url), indent=4, ensure_ascii=False))
driver.close()

{
    "url": "https://dantri.com.vn/xa-hoi/thanh-uy-tphcm-bo-sung-4-uy-vien-ban-thuong-vu-20240821155616419.htm",
    "title": "Thành ủy TPHCM bổ sung 4 ủy viên Ban Thường vụ",
    "description": "Chiều 21/8, Thành ủy TPHCM tổ chức lễ công bố quyết định cán bộ, thống nhất bầu bổ sung 4 người vào ủy viên Ban Thường vụ Thành ủy TPHCM nhiệm kỳ 2020-2025.",
    "content": "Bí thư Thành ủy TPHCM Nguyễn Văn Nên chủ trì buổi lễ công bố quyết định. Cùng dự có Chủ tịch UBND TPHCM Phan Văn Mãi; Chủ tịch HĐND TPHCM Nguyễn Thị Lệ; Phó bí thư Thành ủy, Chủ tịch MTTQ Việt Nam TPHCM Nguyễn Phước Lộc và các ủy viên Ban Thường vụ Thành ủy.\nTheo đó, Ban Bí thư Trung ương Đảng quyết định chuẩn y bốn người tham gia Ban Thường vụ Thành ủy TPHCM khóa XI nhiệm kỳ 2020-2025 gồm: Bà Văn Thị Bạch Tuyết, Trưởng Ban Tổ chức Thành ủy TPHCM; ông Nguyễn Mạnh Cường, Trưởng Ban Dân vận Thành ủy TPHCM; ông Phạm Thành Kiên, Phó chủ tịch HĐND TPHCM; ông Dương Anh Đức, Bí thư Quận ủy quận 1.\nBà Văn Thị Bạch Tuyết, Trưởn

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    cat_data = []
    for url in tqdm(urls, desc=f"Processing {cat}", unit="article"):
        try:
            cat_data.append(get_content_metadata(driver, url))

        # NoSuchElementException
        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue

        except MagazineArticleException as e:
            continue


    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DANTRI_DATA_FOLDER, name_file_cat), "w", encoding='utf-8') as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

Thu thập dữ liệu thể loại PHÁP LUẬT ..


Processing PHÁP LUẬT: 100%|██████████| 402/402 [29:15<00:00,  4.37s/article]


Thu thập dữ liệu thể loại DU LỊCH ..


Processing DU LỊCH: 100%|██████████| 439/439 [47:15<00:00,  6.46s/article]


Thu thập dữ liệu thể loại ĐỜI SỐNG ..


Processing ĐỜI SỐNG:  86%|████████▌ | 359/418 [37:18<06:08,  6.24s/article]

Bug at url: https://dantri.com.vn/doi-song/cuoc-doi-cua-ty-phu-vo-gia-cu-khong-nha-khong-xe-chi-mot-tui-do-cam-tay-20210707215843661.htm, with NoSuchElementException


Processing ĐỜI SỐNG:  99%|█████████▉| 414/418 [43:17<00:17,  4.45s/article]